In [4]:
import numpy as np
import pandas as pd
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [4]:
data = pd.read_csv('news_20171001-20200430')

In [5]:
data.head()

,Unnamed: 0,author,title,description,url,publishedAt,content,from,date
0,0,Samuel Wade,"Katy Perry, Publishers, and Self-censorship in...","On Monday, CDT Chinese reposted a letter circu...",https://chinadigitaltimes.net/2017/11/katy-per...,2017-11-08T02:59:56Z,"On Monday, CDT Chinese reposted a letter circu...",Chinadigitaltimes.net,2017-11-08
1,1,Jesse Hamilton,Bank of Amazon? Regulator Floats Idea of Mergi...,Bloomberg Bank of Amazon? Regulator Floats Ide...,https://www.bloomberg.com/news/articles/2017-1...,2017-11-08T15:07:48Z,Bank of Amazon. Facebook Financial. Wal-Bank. ...,Bloomberg,2017-11-08
2,2,NaN,"Video games could be next for Snapchat, China'...",Chinese gaming and social media company Tencen...,https://japantoday.com/category/tech/update-2-...,2017-11-09T22:24:06Z,Chinese gaming and social media company Tencen...,Japantoday.com,2017-11-09
3,3,Tim Dixon,DiDi’s Huge Role In The Quickly Developing Chi...,Didi Chuxing (China's Uber-like ride-hailing s...,https://cleantechnica.com/2017/11/09/didis-hug...,2017-11-09T23:50:30Z,"Cars Published on November 9th, 2017 | by Tim ...",Cleantechnica.com,2017-11-09
4,4,NaN,Why You Should Buy JD.Com Inc(ADR) Stock Immed...,Shares of JD.com Inc(ADR) (NASDAQ:JD) have set...,https://finance.yahoo.com/news/why-buy-jd-com-...,2017-11-09T17:05:21Z,Shares of JD.com Inc(ADR) (NASDAQ: JD ) have s...,Yahoo.com,2017-11-09


In [9]:
data.count()

Unnamed: 0     5348
author         4542
title          5347
description    5282
url            5348
publishedAt    5348
content        4994
from           5348
date           5348
dtype: int64

## Using text blob

In [10]:
date = list()
title_pos = list()
title_neg = list()
desc_pos = list()
desc_neg = list()
content_pos = list()
content_neg = list()

In [92]:
def textblob_opinion(text_content, pos_count_container, neg_count_container):
    opinion = TextBlob(text_content, analyzer=NaiveBayesAnalyzer())
    
    if opinion.sentiment.classification == 'pos':
        pos_count_container.append(1)
        neg_count_container.append(0)
        
    elif opinion.sentiment.classification == 'neg':
        pos_count_container.append(0)
        neg_count_container.append(1)
        
    else:
        pos_count_container.append(0)
        neg_count_container.append(0)


for index, row in data.iterrows():
    date.append(row["date"])
    
    if type(row["title"]) == str:
        textblob_opinion(row["title"], title_pos, title_neg)    
    else:
        title_pos.append(0)
        title_neg.append(0)
        
    if type(row["description"]) == str:
        textblob_opinion(row["description"], desc_pos, desc_neg)
    else:
        desc_pos.append(0)
        desc_neg.append(0)
            
    if type(row["content"]) == str:
        textblob_opinion(row["content"], content_pos, content_neg)
    else:
        content_pos.append(0)
        content_neg.append(0)
    

In [94]:
d = {"date": date, "title_pos": title_pos, "title_neg": title_neg, "desc_pos": desc_pos, "desc_neg": desc_neg, 
    "content_pos": content_pos, "content_neg": content_neg}

In [1]:
#tb_df_csv = tb_df.to_csv("sentiment_analysis.csv")

## Reading from sentiment_analysis.csv which stores the senitment analysis results of the data. for further analysis of the data is very expensive operation to do be all over again.

In [5]:
tb_df = pd.read_csv("sentiment_analysis.csv")

In [6]:
date = tb_df.date
title_pos = tb_df.title_pos
title_neg = tb_df.title_neg
desc_pos = tb_df.desc_pos
desc_neg = tb_df.desc_neg
content_pos = tb_df.content_pos
content_neg = tb_df.content_neg

In [7]:
d = {"date": date, "title_pos": title_pos, "title_neg": title_neg, "desc_pos": desc_pos, "desc_neg": desc_neg, 
    "content_pos": content_pos, "content_neg": content_neg}

In [8]:
unique_dates = list()

for dt in d["date"]:
    if dt not in unique_dates:
        unique_dates.append(dt)

In [9]:
date_index_dict = {dt: list() for dt in unique_dates}

In [10]:
for u_date in unique_dates:
    for idx in range(len(d["date"])):
        if u_date == d["date"][idx]:
           date_index_dict[u_date].append(idx)

In [11]:
unq_title_pos = list()
unq_title_neg = list()
unq_desc_pos = list()
unq_desc_neg = list()
unq_content_pos = list()
unq_content_neg = list()

In [12]:
for dt in date_index_dict:
    t_pos = t_neg = c_pos = c_neg = d_pos = d_neg = 0
    
    for idx in date_index_dict[dt]:
        t_pos += title_pos[idx] 
        t_neg += title_neg[idx] 
        d_pos += desc_pos[idx] 
        d_neg += desc_neg[idx]
        c_pos += content_pos[idx]
        c_neg += content_neg[idx]
    
    unq_title_pos.append(t_pos)
    unq_title_neg.append(t_neg)
    unq_desc_pos.append(d_pos)
    unq_desc_neg.append(d_neg)
    unq_content_pos.append(c_pos)
    unq_content_neg.append(c_neg)
        

In [13]:
unq_d = {
     "date": unique_dates, 
     "title_pos": unq_title_pos, 
     "title_neg": unq_title_neg, 
     "desc_pos": unq_desc_pos, 
     "desc_neg": unq_desc_neg, 
     "content_pos": unq_content_pos, 
     "content_neg": unq_content_neg
}

In [14]:
new_tb_df = pd.DataFrame(unq_d)

In [15]:
new_tb_df.head()

,date,title_pos,title_neg,desc_pos,desc_neg,content_pos,content_neg
0,2017-11-08,1,1,2,0,2,0
1,2017-11-09,1,3,4,0,4,0
2,2017-11-10,5,2,5,2,7,0
3,2017-11-11,0,4,4,0,4,0
4,2017-11-12,4,1,4,1,4,0


In [23]:
new_tb_df.shape

(905, 7)

In [28]:
x = new_tb_df.drop("date", axis=1)
x = x.div(new_tb_df.sum(axis=1), axis=0)
x.head()

,title_pos,title_neg,desc_pos,desc_neg,content_pos,content_neg
0,0.166667,0.166667,0.333333,0.000000,0.333333,0.0
1,0.083333,0.250000,0.333333,0.000000,0.333333,0.0
2,0.238095,0.095238,0.238095,0.095238,0.333333,0.0
3,0.000000,0.333333,0.333333,0.000000,0.333333,0.0
4,0.285714,0.071429,0.285714,0.071429,0.285714,0.0


In [29]:
x['date'] = unique_dates

In [30]:
cols = list(unq_d.keys())
x.reindex(columns=cols)

,date,title_pos,title_neg,desc_pos,desc_neg,content_pos,content_neg
0,2017-11-08,0.166667,0.166667,0.333333,0.000000,0.333333,0.000000
1,2017-11-09,0.083333,0.250000,0.333333,0.000000,0.333333,0.000000
2,2017-11-10,0.238095,0.095238,0.238095,0.095238,0.333333,0.000000
3,2017-11-11,0.000000,0.333333,0.333333,0.000000,0.333333,0.000000
4,2017-11-12,0.285714,0.071429,0.285714,0.071429,0.285714,0.000000
...,...,...,...,...,...,...,...
900,2020-04-26,0.266667,0.066667,0.266667,0.066667,0.333333,0.000000
901,2020-04-27,0.266667,0.066667,0.266667,0.066667,0.333333,0.000000
902,2020-04-28,0.294118,0.058824,0.352941,0.000000,0.294118,0.000000
903,2020-04-29,0.142857,0.214286,0.357143,0.000000,0.285714,0.000000


In [34]:
x_csv = x.to_csv("norma_sentiment_analysis.csv")

## nltk sentiment analyzer

In [14]:
sid = SentimentIntensityAnalyzer()

In [47]:
print(data.description[150])
print(sid.polarity_scores(data.content[150]))

Some of its greatest advances depend heavily on local conditions.
{'neg': 0.039, 'neu': 0.852, 'pos': 0.11, 'compound': 0.9969}
